In [1]:
import pandas as pd
import numpy as np

import warnings

# 모든 종류의 경고를 무시하도록 설정
warnings.filterwarnings("ignore")

In [2]:
# HoBo
df_oo = pd.read_csv('/content/drive/MyDrive/ch5. 소득예측/데이터 확인 과정/train_hobxsd.csv')

# test = pd.read_csv('/content/drive/MyDrive/ch5. 소득예측/test_oom.csv')

In [3]:
# 함수 만들기
def label_encoder(df) :
  # 라이브러리 불러오기
  from sklearn.preprocessing import LabelEncoder
  # 인코딩할 칼럼 추출
  df_categorical = df.select_dtypes(include=['object']).columns
  # 식별자 ID 제외하기
  df_categorical = df_categorical.drop('ID')
  # 라벨 인코딩 적용
  label_encoder = LabelEncoder()
  df[df_categorical] = df[df_categorical].apply(label_encoder.fit_transform)

In [4]:
label_encoder(df_oo)

In [5]:
# # # features와 label 분리 작업 + 연령대로 머신러닝
X_features = df_oo.loc[:,(df_oo.columns != 'ID') & (df_oo.columns != 'Income')]
y_label = df_oo['Income']

In [6]:
df_col = [2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 18, 20, 21, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]
for i in df_col :
  df_oo.iloc[:,i] = df_oo.iloc[:,i].astype('category')

In [7]:
# train_ test분리
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_features, y_label, test_size = 0.2, random_state = 42)

In [8]:
# categorical_features_indices = [1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 17, 18, 19, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
# for i in categorical_features_indices :
#   X_train.iloc[:,i] = X_train.iloc[:,i].astype('category')
#   X_test.iloc[:,i] = X_test.iloc[:,i].astype('category')

In [9]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error
import numpy as np

# RMSE 계산
def calculate_rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# MAE 계산
def calculate_mae(y_true, y_pred):
    return mean_absolute_error(y_true, y_pred)

# MDAPE 계산
def calculate_mdape(y_true, y_pred):
    absolute_percentage_errors = np.abs((y_true - y_pred) / y_true)
    return np.median(absolute_percentage_errors) * 100

# MAPE 계산
def calculate_mape(y_true, y_pred):
    absolute_percentage_errors = np.abs((y_true - y_pred) / y_true)
    return np.mean(absolute_percentage_errors) * 100

## Lgbm

In [10]:
from lightgbm import LGBMRegressor
import lightgbm as lgb

In [11]:
# 초기화된 모든 특성을 포함한 특성 리스트를 만듭니다.
selected_features = list(X_train.columns)

model = LGBMRegressor()
best_score = float('inf')

# Backward Elimination
while len(selected_features) > 0:
    scores = []
    for feature in selected_features:
        features_to_use = selected_features.copy()
        features_to_use.remove(feature)

        model.fit(X_train.loc[:, features_to_use], y_train)
        y_pred = model.predict(X_test.loc[:, features_to_use])
        score = np.sqrt(mean_squared_error(y_test, y_pred))

        scores.append((score, feature))

    scores.sort()
    best_candidate_score, best_candidate_feature = scores[0]

    if best_candidate_score < best_score:
        selected_features.remove(best_candidate_feature)
        best_score = best_candidate_score
    else:
        break

print("Selected Features:", selected_features)
print("Best RMSE:", best_score)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005569 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1135
[LightGBM] [Info] Number of data points in the train set: 16000, number of used features: 31
[LightGBM] [Info] Start training from score 556.153500
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005569 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1225
[LightGBM] [Info] Number of data points in the train set: 16000, number of used features: 31
[LightGBM] [Info] Start training from score 556.153500
[LightGBM] [Warning] Fou

In [12]:
model_lgbm_final = lgb.LGBMRegressor()
model_lgbm_final.fit(X_train.loc[:, selected_features], y_train)
y_pred_final = model_lgbm_final.predict(X_test.loc[:, selected_features])
final_rmse = np.sqrt(mean_squared_error(y_test, y_pred_final))
final_mdape = np.median(np.abs((y_test - y_pred_final) / y_test)) * 100

print("Final RMSE with selected features:", final_rmse)
print("Final MDAPE with selected features:", final_mdape)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004846 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1169
[LightGBM] [Info] Number of data points in the train set: 16000, number of used features: 28
[LightGBM] [Info] Start training from score 556.153500
Final RMSE with selected features: 588.0386604172658
Final MDAPE with selected features: 57.00717160816355


In [13]:
# !pip install catboost

In [14]:
import xgboost as xgb

# 경고 메시지 출력을 숨기는 함수 정의
def suppress_xgboost_warnings():
    xgb.set_config(verbosity=0)

# suppress_xgboost_warnings 함수 호출
suppress_xgboost_warnings()

# 최적의 모델 및 점수 추적을 위한 변수 초기화
best_rmse = float('inf')
best_model = None
selected_features = list(X_train.columns)  # 모든 특성을 포함한 상태로 시작

# 후진소거법
while len(selected_features) > 1:
    scores = []
    for feature in selected_features:
        features_to_use = selected_features.copy()
        features_to_use.remove(feature)  # 특성을 제거한 후의 특성 리스트

        # 특성 선택에 따른 데이터셋 생성
        dtrain = xgb.DMatrix(X_train.loc[:, features_to_use], label=y_train, enable_categorical=True)
        dtest = xgb.DMatrix(X_test.loc[:, features_to_use], enable_categorical=True)

        # XGBoost 모델 훈련
        model = xgb.train({}, dtrain)

        # 테스트 데이터로 예측
        predictions = model.predict(dtest)

        # 평가 지표 계산
        rmse = calculate_rmse(y_test, predictions)
        scores.append((rmse, feature))

    # 특성별 점수 정렬
    scores.sort()

    # 가장 좋은 특성 선택
    best_candidate_rmse, worst_feature = scores[0]

    # 현재까지의 최적 모델 및 점수 갱신
    if best_candidate_rmse < best_rmse:
        selected_features.remove(worst_feature)
        best_rmse = best_candidate_rmse
    else:
        break

# 선택된 특성의 변수 이름 출력
print("Selected Features:", features_to_use)
print("Best RMSE:", best_rmse)

Selected Features: ['Age', 'Gender', 'Education_Status', 'Employment_Status', 'Working_Week (Yearly)', 'Industry_Status', 'Occupation_Status', 'Race', 'Hispanic_Origin', 'Martial_Status', 'Citizenship', 'Tax_Status', 'Gains', 'Income_Status', 'Us_ox', 'Age_range', 'Age_agegender', 'Employment_Group2', 'Martial_group', 'Native_yn', 'Allother_yn']
Best RMSE: 587.5852112582658


In [15]:
import xgboost as xgb


# XGBoost 모델 초기화하기
model = xgb.XGBRegressor()

# 모델 훈련
dtrain = xgb.DMatrix(X_train.loc[:, features_to_use], label=y_train, enable_categorical=True)
model = xgb.train({}, dtrain)

# 테스트 데이터로 예측
dtest = xgb.DMatrix(X_test.loc[:, features_to_use], enable_categorical=True)
predictions = model.predict(dtest)

# 평가 지표 계산
rmse = calculate_rmse(y_test, predictions)
mae = calculate_mae(y_test, predictions)
mdape = calculate_mdape(y_test, predictions)
mape = calculate_mape(y_test, predictions)

print("RMSE:", rmse)
print("MAE:", mae)
print("MDAPE:", mdape)
print("MAPE:", mape)

RMSE: 587.5852112582658
MAE: 334.38147844314574
MDAPE: 56.03914005596915
MAPE: inf
